In [1]:
import higlass as hg
import cooler
import glob

In [2]:
coords = "chr1:44,000,000-68,000,000"
coolers = list(sorted(glob.glob("../../data/output/nfcore_hic/mcools/hg38_00?_MCF10A*.mcool")))

tads2d_10k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_10000_2D.beddb"))
)
tads2d_20k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_20000_2D.beddb"))
)
tads2d_50k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_50000_2D.beddb"))
)
tads2d_100k = list(
    sorted(glob.glob("../../data/output/tad_analysis/clodius/hg38_00?_MCF10A*_domains_ICE_100000_2D.beddb"))
)

In [3]:
def compute_domain(coords, clr: cooler.Cooler, padding=0.0):
    df = clr.matrix(as_pixels=True).fetch(coords)

    pos1 = df["bin1_id"].min() * clr.binsize
    pos2 = df["bin1_id"].max() * clr.binsize

    if padding != 0.0:
        extent = pos2 - pos1
        new_extent = int(padding * extent)
        pos1 -= (new_extent - extent) // 2
        pos2 += (new_extent - extent) // 2

    return max(0, pos1), pos2

## TADs as 2D domains

In [4]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

cooler_tiles = {}
tad_tiles = {}
cooler_tracks = {}

for bedpe, clr in zip(tads2d_50k, coolers):
    cooler_tiles[clr] = hg.cooler(clr)
    tad_tiles[clr] = hg.bed2ddb(bedpe)
    cooler_tracks[clr] = hg.combine(
        cooler_tiles[clr].track("linear-heatmap"), tad_tiles[clr].track("linear-2d-rectangle-domains"), height=70
    )


chrom_sizes_track = list(cooler_tiles.values())[0].track("chromosome-labels")
ga_track = gene_annotation_tile.track("horizontal-gene-annotations")
top_track = hg.combine(ga_track, chrom_sizes_track, height=150)

uri = coolers[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"
domain = compute_domain(coords, cooler.Cooler(uri))

tracks = [(top_track, "top")]

for key, track in cooler_tracks.items():
    tracks.append((track, "top"))

view = hg.view(*tracks).domain(x=domain, y=domain)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[CombinedTrack(type='combined', uid='79765370-0410-4cba-9e21-a896b3932e0f', width=None, height=150, options=None, contents=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='32e9209e-e051-4621-85cc-3697803ad8af', width=None, height=None, options={'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:46017/tilesets/api/v1/', type='chromosome-labels', uid='50406c3b-ad69-47ca-bb99-80293c464fa1', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='ee2a1699-df71-4b63-b46f-84a3ddd0700b', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='b8626320-07b0-4b5e-a21c-279edd5299be', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='37b59a41512dd7b29a8e23ee9faa94fe', server='http://localhost:46017/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='ea8d34d8-a842-4474-9093-5f349fda69f5', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='26046171-7789-4b84-8188-7814727261b1', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='ad01de40dfecfd01d435287c167bd33b', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='a53f30aa-9c05-4257-a4fe-bbcbf70ff72e', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d99d8524e24840c79a514cbddd5b477b', server='http://localhost:46017/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='0e4c15db-0299-4487-a9ea-461496000f4e', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='a8294b86-b10f-49d7-924c-54c0a5f0216f', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='335f43c2893812a8f8d92ac88375b18f', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='841ac498-9472-4f94-b0cf-cb8b46403a14', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='7837de292fddf348e6808d9f8a7be466', server='http://localhost:46017/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='df5a462a-9250-4a1e-8dfc-9aa84ceeea75', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='090a7705-3bb6-4578-9f2a-9973fd8e7a2c', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='4a8de3b966951cb85def4c4d9c1d2d7a', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='80d25188-2a16-4f43-ab5d-5d4780171729', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='3bf166c7be1edd41440cf7a8b608b4c9', server='http://localhost:46017/tilesets/api/v1/', type='linear-2d-rectangle-domains', uid='3c322dc4-3a84-404e-abdb-bcc4830a6b3f', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), CombinedTrack(type='combined', uid='47f5debd-4708-490d-9726-11b4b46fbef0', width=None, height=70, options=None, contents=[EnumTrack(tilesetUid='1b9842a1ee691f500e75a7134e91497e', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='f97a1ad4-4fdd-4a8e-b347-2a591e89251b', width=None, height=None, opti

## 50 kbp TADs as BED tracks

In [5]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

cooler_tiles = {}
tad_tiles = {}
cooler_tracks = {}
tad_tracks = {}

for bed50k, clr in zip(tads2d_50k, coolers):
    cooler_tiles[clr] = hg.cooler(clr)

    cooler_tracks[clr] = cooler_tiles[clr].track("linear-heatmap", height=75)
    tad_tracks[clr] = [hg.bed2ddb(bed).track("bedlike", height=20).opts(annotationStyle="segment") for bed in (bed50k,)]

chrom_sizes_track = list(cooler_tiles.values())[0].track("chromosome-labels")
ga_track = gene_annotation_tile.track("horizontal-gene-annotations")
top_track = hg.combine(ga_track, chrom_sizes_track, height=150)

uri = coolers[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"
domain = compute_domain(coords, cooler.Cooler(uri))

views = {}

tracks = [(top_track, "top")]

for tad_tracks, clr_track in zip(tad_tracks.values(), cooler_tracks.values()):
    tracks.append((clr_track, "top"))
    for trk in tad_tracks:
        tracks.append((trk, "top"))

view = hg.view(*tracks).domain(x=domain, y=domain)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[CombinedTrack(type='combined', uid='9247976e-20cf-4e12-a5ce-3b6684f543f6', width=None, height=150, options=None, contents=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='5b96a563-ea71-4a1c-95eb-3ac802e4c6af', width=None, height=None, options={'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:46017/tilesets/api/v1/', type='chromosome-labels', uid='f543627d-ed5b-4ea3-a30f-716e78f98852', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], position=None), EnumTrack(tilesetUid='691b56279e5f36cab017e040e7dd4bf2', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='04b63ce1-061b-45a5-9bbf-4eddf59102fb', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='37b59a41512dd7b29a8e23ee9faa94fe', server='http://localhost:46017/tilesets/api/v1/', type='bedlike', uid='2ac165de-d37b-4726-aaaa-ee6f038858ae', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='ad01de40dfecfd01d435287c167bd33b', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='e58da063-70d6-4236-8c17-a3edd6070460', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='d99d8524e24840c79a514cbddd5b477b', server='http://localhost:46017/tilesets/api/v1/', type='bedlike', uid='2812a5ad-5a27-46b3-82f0-17231c2491d2', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='335f43c2893812a8f8d92ac88375b18f', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='4a258ca9-e507-4d6d-ad53-7b63742ead6c', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='7837de292fddf348e6808d9f8a7be466', server='http://localhost:46017/tilesets/api/v1/', type='bedlike', uid='033d18e1-d749-4ea4-a432-ae3a5fd47823', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='4a8de3b966951cb85def4c4d9c1d2d7a', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='3a1af7f6-2d7d-48a2-b263-c3214f831273', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='3bf166c7be1edd41440cf7a8b608b4c9', server='http://localhost:46017/tilesets/api/v1/', type='bedlike', uid='297104ec-875a-42d0-8f91-d632c7f791a4', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='1b9842a1ee691f500e75a7134e91497e', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='03261e03-83b6-45b7-8a78-4169d83cc07b', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='2fe00402c47abf1d38ac3b726730ed6a', server='http://localhost:46017/tilesets/api/v1/', type='bedlike', uid='db02c941-8763-4898-927c-9fa353972573', width=None, height=20, options={'annotationStyle': 'segment'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='249786704855055fb4f35174ca507bf2', server='http://localhost:46017/tilesets/api/v1/', type='linear-heatmap', uid='05ac6959-5231-4601-a7a9-fc4b5e5b448f', width=None, height=75, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='7